# Installing Mongodb

In [1]:
!wget -qO - https://www.mongodb.org/static/pgp/server-6.0.asc | sudo apt-key add -
!echo "deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu focal/mongodb-org/6.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list
!sudo apt-get update > /dev/null
!sudo apt-get -qq install -y mongodb-org > /dev/null
!mkdir -p /data/db 
!mkdir -p /data/db/log

OK
deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu focal/mongodb-org/6.0 multiverse
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 9.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


# Running Mongodb

In order to run Mongodb, the mongod server must first be run. But, running a process in background in jupyter doesn't seem easy. A quick fix is to just use python's os module and use the & character to run mongod in background. (For some reason, colab doesn't recognize mongod service).

In [2]:
import os
os.system('mongod &')

0

It may not look like it, but the mongodb process is running in the background. We can verify by querying the port 27017. There will be a message that says MongoDB is being accessed over HTTP. But it at least shows that mongodb is up.

In [3]:
!curl localhost:27017

It looks like you are trying to access MongoDB over HTTP on the native driver port.


#Installing pymongo

A standard client to use with MongoDB is pymongo in python.

In [4]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


We have installed pymongo. The basic code that follows is a part of the standard pymongo tutorial. Nevertheless, it's beneficial to execute and see by ourselves.

# Creating a MongoDB Connection

In [5]:
from pymongo import MongoClient
client = MongoClient()
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

We got a valid connection which means everything's working till now. Let's work with a database now. Let's go ahead a create a new one.

In [6]:
db = client.test_database #or client['test_database']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_database')

Let's create a new collection in the database. (Think of collections as tables).

In [7]:
posts = db.posts  # or db['posts'] ... if the collection doesn't exist, mongodb creates one for you.

Let's insert an item. Remember that Mongodb documents are jsons.

In [8]:
import datetime
post = {"author": "Mike",
        "text": "My first blog post!",
        "tags": ["mongodb", "python", "pymongo"],
        "date": datetime.datetime.utcnow()}
post

{'author': 'Mike',
 'text': 'My first blog post!',
 'tags': ['mongodb', 'python', 'pymongo'],
 'date': datetime.datetime(2023, 2, 15, 3, 33, 21, 293166)}

We can use `insert_one` method to insert a document.

In [9]:
insert = posts.insert_one(post)
insert.inserted_id

ObjectId('63ec52810fd9afa45ffc6c77')

You can now see that a collection is present in our database.

In [10]:
db.list_collection_names()

['posts']

Let's insert a few more documents. We can insert many documents by using a `bulk_insert` method.

In [11]:
post1 = {"author": "Socrates",
        "text": "I know nothing!",
        "tags": ["philosophy", "greek", "pymongo"],
        "date": datetime.datetime.utcnow()}

post2 = {"author": "Plato",
        "text": "If you lie, at least make it noble!",
        "tags": ["socrates", "lies", "pymongo"],
        "date": datetime.datetime.utcnow()}

post3 = {"author": "Pravesh",
        "text": "Greek Philosophy confuses me!",
        "tags": ["greek", "study", "boring"],
        "date": datetime.datetime.utcnow()}

post4 = {"author": "Pravesh",
        "text": "Greek Philosophy confuses me a lot!",
        "tags": ["greek", "study", "tired"],
        "date": datetime.datetime.utcnow()}

all_posts = [post1, post2, post3, post4]
many_inserts = posts.insert_many(all_posts)
many_inserts.inserted_ids

[ObjectId('63ec52810fd9afa45ffc6c78'),
 ObjectId('63ec52810fd9afa45ffc6c79'),
 ObjectId('63ec52810fd9afa45ffc6c7a'),
 ObjectId('63ec52810fd9afa45ffc6c7b')]

Querying a single document can be done using a `find_one` command.

In [12]:
posts.find_one()

{'_id': ObjectId('63ec52810fd9afa45ffc6c77'),
 'author': 'Mike',
 'text': 'My first blog post!',
 'tags': ['mongodb', 'python', 'pymongo'],
 'date': datetime.datetime(2023, 2, 15, 3, 33, 21, 293000)}

It just gave us the first document. We can also retrieve posts by a specific author.

In [13]:
posts.find_one({"author": "Plato"})

{'_id': ObjectId('63ec52810fd9afa45ffc6c79'),
 'author': 'Plato',
 'text': 'If you lie, at least make it noble!',
 'tags': ['socrates', 'lies', 'pymongo'],
 'date': datetime.datetime(2023, 2, 15, 3, 33, 21, 374000)}

If we try to find non-existent records, we will not get any results at all!

In [14]:
posts.find_one({"author": "Diogenes"})

We can find a document by using its object id as well. For example, in the insert above, Socrates' object id was 63e67354a8e8ba18f1038690. Let's go ahead and use it.

In [15]:
posts.find_one({"_id": "63e67354a8e8ba18f1038690"})

No result! Unfortunately, MongoDB has a special representation for objectID. We can't just use strings to query the id.

In [16]:
from bson.objectid import ObjectId
posts.find_one({"_id": ObjectId("63e67354a8e8ba18f1038690")})

Found it! BSON is an extended version of JSON that supports Binary objects.

We can query for more than one documents as well.

In [17]:
result = posts.find({"author": "Pravesh"})
result

However, the result of this operation is not a python dictionary like the one we had before. This is because a find operation may result in thousands and thousands of records. You wouldn't want to download all of them, as it may end up taking you lots of time! Instead, you can iterate over a cursor in multiple ways.

In [18]:
for c in result:
  print (c)

{'_id': ObjectId('63ec52810fd9afa45ffc6c7a'), 'author': 'Pravesh', 'text': 'Greek Philosophy confuses me!', 'tags': ['greek', 'study', 'boring'], 'date': datetime.datetime(2023, 2, 15, 3, 33, 21, 374000)}
{'_id': ObjectId('63ec52810fd9afa45ffc6c7b'), 'author': 'Pravesh', 'text': 'Greek Philosophy confuses me a lot!', 'tags': ['greek', 'study', 'tired'], 'date': datetime.datetime(2023, 2, 15, 3, 33, 21, 374000)}


Alternatively, you can also just create a list out of it. You'll have to query it once more before doing that becase the iterator right now is empty.

In [19]:
result = posts.find({"author": "Pravesh"})
l = list(result)
l

[{'_id': ObjectId('63ec52810fd9afa45ffc6c7a'),
  'author': 'Pravesh',
  'text': 'Greek Philosophy confuses me!',
  'tags': ['greek', 'study', 'boring'],
  'date': datetime.datetime(2023, 2, 15, 3, 33, 21, 374000)},
 {'_id': ObjectId('63ec52810fd9afa45ffc6c7b'),
  'author': 'Pravesh',
  'text': 'Greek Philosophy confuses me a lot!',
  'tags': ['greek', 'study', 'tired'],
  'date': datetime.datetime(2023, 2, 15, 3, 33, 21, 374000)}]

See, now it's easier to deal with.

Let's count how many documents we have made till now.

In [20]:
posts.count_documents({})

5

You can also count by specific fields.

In [21]:
posts.count_documents({"author": "Pravesh"})

2

#Updating and Deleting

Like before, you can update either one document, or multiple documents. Let's start with update one. This method will update the first document it encounters. If you pass an empty filter, then that would mean that it'd update the first document of the collection. If you pass a non-empty filter, it'd update the first result.

In [22]:
update = posts.update_one({"author": "Pravesh"}, # filter. Could just be empty as well i.e. {}
                          {"$set": {"text": "I love Greek Philosophy!"}})
update

Let's see if we have updated results!

In [23]:
list(posts.find({"author": "Pravesh"}))

[{'_id': ObjectId('63ec52810fd9afa45ffc6c7a'),
  'author': 'Pravesh',
  'text': 'I love Greek Philosophy!',
  'tags': ['greek', 'study', 'boring'],
  'date': datetime.datetime(2023, 2, 15, 3, 33, 21, 374000)},
 {'_id': ObjectId('63ec52810fd9afa45ffc6c7b'),
  'author': 'Pravesh',
  'text': 'Greek Philosophy confuses me a lot!',
  'tags': ['greek', 'study', 'tired'],
  'date': datetime.datetime(2023, 2, 15, 3, 33, 21, 374000)}]

Success. Now we can go ahead and update many records at once as well.

In [24]:
update = posts.update_many({"author": "Pravesh"}, # filter. Could just be empty as well i.e. {}
                          {"$set": {"text": "I absolutely love Greek Philosophy!!!"}})
list(posts.find({"author": "Pravesh"}))

[{'_id': ObjectId('63ec52810fd9afa45ffc6c7a'),
  'author': 'Pravesh',
  'text': 'I absolutely love Greek Philosophy!!!',
  'tags': ['greek', 'study', 'boring'],
  'date': datetime.datetime(2023, 2, 15, 3, 33, 21, 374000)},
 {'_id': ObjectId('63ec52810fd9afa45ffc6c7b'),
  'author': 'Pravesh',
  'text': 'I absolutely love Greek Philosophy!!!',
  'tags': ['greek', 'study', 'tired'],
  'date': datetime.datetime(2023, 2, 15, 3, 33, 21, 374000)}]

As you can see, the posts have been updated and apparently, I now love greek philosophy.

Now we can go ahead and delete stuffs. The deletion works the same was as update. You have a filter which you use to decide which documents to delete and you either use a delete_one or delete_many function depending upon the numbers of document you want to delete. In our case, let's go ahead and delete everything that's posted by either Plato or Socrates. This would involve a complex query.

So before we delete, let's verify if we can obtain the documents that we do want to delete.

In [25]:
list(posts.find({"$or": [{"author": "Plato"}, {"author": "Socrates"}]}))

[{'_id': ObjectId('63ec52810fd9afa45ffc6c78'),
  'author': 'Socrates',
  'text': 'I know nothing!',
  'tags': ['philosophy', 'greek', 'pymongo'],
  'date': datetime.datetime(2023, 2, 15, 3, 33, 21, 374000)},
 {'_id': ObjectId('63ec52810fd9afa45ffc6c79'),
  'author': 'Plato',
  'text': 'If you lie, at least make it noble!',
  'tags': ['socrates', 'lies', 'pymongo'],
  'date': datetime.datetime(2023, 2, 15, 3, 33, 21, 374000)}]

So we can obtain the records. Now we use the same filter to delete many documents at once using the delete_many function.

In [26]:
posts.delete_many({"$or": [{"author": "Plato"}, {"author": "Socrates"}]})

To verify that our documents have been deleted, let's count our documents again. We know that we started out with 5 documents. Let's see how many remain.

In [27]:
posts.count_documents({})

3

Success! We have successfully deleted the two documents. 


# Aggregation Pipeline

Aggregation pipelines in MongoDB are used to do complex queries, joins, and aggregations. The example below is based on the one from official Mongo documentation.

Let's create a new collection and instantiate it with a few documents.

In [28]:
ISODate = lambda s: datetime.datetime.strptime(s, "%Y-%m-%dT%H:%M:%SZ")

In [29]:
orders = db.orders
inserts = orders.insert_many([
   { "_id": 0, "name": "Pepperoni", "size": "small", "price": 19,
     "quantity": 10, "date": ISODate( "2021-03-13T08:14:30Z" ) },
   { "_id": 1, "name": "Pepperoni", "size": "medium", "price": 20,
     "quantity": 20, "date" : ISODate( "2021-03-13T09:13:24Z" ) },
   { "_id": 2, "name": "Pepperoni", "size": "large", "price": 21,
     "quantity": 30, "date" : ISODate( "2021-03-17T09:22:12Z" ) },
   { "_id": 3, "name": "Cheese", "size": "small", "price": 12,
     "quantity": 15, "date" : ISODate( "2021-03-13T11:21:39Z" ) },
   { "_id": 4, "name": "Cheese", "size": "medium", "price": 13,
     "quantity": 50, "date" : ISODate( "2022-01-12T21:23:13Z" ) },
   { "_id": 5, "name": "Cheese", "size": "large", "price": 14,
     "quantity": 10, "date" : ISODate( "2022-01-12T05:08:13Z" ) },
   { "_id": 6, "name": "Vegan", "size": "small", "price": 17,
     "quantity": 10, "date" : ISODate( "2021-01-13T05:08:13Z" ) },
   { "_id": 7, "name": "Vegan", "size": "medium", "price": 18,
     "quantity": 10, "date" : ISODate( "2021-01-13T05:10:13Z" ) }
] )
inserts.inserted_ids

[0, 1, 2, 3, 4, 5, 6, 7]

We can obtain specific columns of the collections that fulfill certain criteria by using match and project operators. For example, to obtain all records of Vegan small size pizza, we could do:

In [30]:
result = orders.aggregate([
    # The first stage of aggregation is a match query that filters your document and gives you a subset of it.
    # you can use complex queries here as well using operators such as $and, $or etc
    {"$match": {"size": "small", "name": "Vegan"}},
    # The second stage of aggregation i.e. project lets you specify what results to return.
    {"$project": {"_id":1, "name": 1, "date":1, "size":1}}  # specify 1 to obtain the column. Excluded columns are not shown.
])
list(result)

[{'_id': 6,
  'name': 'Vegan',
  'size': 'small',
  'date': datetime.datetime(2021, 1, 13, 5, 8, 13)}]

We can also use project operator to rename columns and do simple mathematical operations on the columns. For example, let's rename the `name` field to `type` and let's obtain the price of the pizza in `cents` instead of dollars.

In [31]:
result = orders.aggregate([
    {"$project": {"_id":1, "type": "$name", "date":1, "size":1, 
                  "price": {"$multiply": ["$price", 100]}}} # select name as type, price*100 as price ... FROM ... 
])
list(result)

[{'_id': 0,
  'size': 'small',
  'date': datetime.datetime(2021, 3, 13, 8, 14, 30),
  'type': 'Pepperoni',
  'price': 1900},
 {'_id': 1,
  'size': 'medium',
  'date': datetime.datetime(2021, 3, 13, 9, 13, 24),
  'type': 'Pepperoni',
  'price': 2000},
 {'_id': 2,
  'size': 'large',
  'date': datetime.datetime(2021, 3, 17, 9, 22, 12),
  'type': 'Pepperoni',
  'price': 2100},
 {'_id': 3,
  'size': 'small',
  'date': datetime.datetime(2021, 3, 13, 11, 21, 39),
  'type': 'Cheese',
  'price': 1200},
 {'_id': 4,
  'size': 'medium',
  'date': datetime.datetime(2022, 1, 12, 21, 23, 13),
  'type': 'Cheese',
  'price': 1300},
 {'_id': 5,
  'size': 'large',
  'date': datetime.datetime(2022, 1, 12, 5, 8, 13),
  'type': 'Cheese',
  'price': 1400},
 {'_id': 6,
  'size': 'small',
  'date': datetime.datetime(2021, 1, 13, 5, 8, 13),
  'type': 'Vegan',
  'price': 1700},
 {'_id': 7,
  'size': 'medium',
  'date': datetime.datetime(2021, 1, 13, 5, 10, 13),
  'type': 'Vegan',
  'price': 1800}]

We can now use aggregations. To calculate the total order quantity for medium sized pizzas as follows:

In [32]:
result = orders.aggregate([
    # The first stage of aggregation is a match query that filters your document and gives you a subset of it.
    # you can use complex queries here as well using operators such as $and, $or etc
    {"$match": {"size": "medium"}},

    # This stage of aggregation is what's called a group. Here we reduce the collection to summary statistics.
    {"$group": {"_id": "$name", "totalQuantity": {"$sum": "$quantity"}}}
])
list(result)

[{'_id': 'Pepperoni', 'totalQuantity': 20},
 {'_id': 'Cheese', 'totalQuantity': 50},
 {'_id': 'Vegan', 'totalQuantity': 10}]

The query above is similar in spirit to the SQL statement `Select name, sum(quantity) from orders where size='medium'`

The next example calculates the total pizza order value and average order quantity of Pepperoni pizzas and the total number of pizza orders.

In [33]:
result = orders.aggregate([
    # Select all pepperoni pizzas
    {"$match": {"name": "Pepperoni"}},

    # Calculate total order and average order
    {"$group": {
                "_id": "$name", 
                "totalQuantity": {"$sum": "$quantity"}, 
                "avgQuantity": {"$avg": "$quantity"},
                "countRecords": {"$sum": 1}}
     }  
])
list(result)

[{'_id': 'Pepperoni',
  'totalQuantity': 60,
  'avgQuantity': 20.0,
  'countRecords': 3}]

In addition to summary statistics, you can also do advance data filter and retrieval using different aggregation operators. Consider if you wanted to find the per pizza price for all orders.

In [34]:
result = orders.aggregate([
    {"$match": {}}, # match is going to be empty because you want to match everything

    {"$project": {"quantity": 1, "price": 1, "per_pizza": {"$divide": ["$quantity", "$price"]}}}   #per_pizza cost = quantity/price
    
])
list(result)

[{'_id': 0, 'price': 19, 'quantity': 10, 'per_pizza': 0.5263157894736842},
 {'_id': 1, 'price': 20, 'quantity': 20, 'per_pizza': 1.0},
 {'_id': 2, 'price': 21, 'quantity': 30, 'per_pizza': 1.4285714285714286},
 {'_id': 3, 'price': 12, 'quantity': 15, 'per_pizza': 1.25},
 {'_id': 4, 'price': 13, 'quantity': 50, 'per_pizza': 3.8461538461538463},
 {'_id': 5, 'price': 14, 'quantity': 10, 'per_pizza': 0.7142857142857143},
 {'_id': 6, 'price': 17, 'quantity': 10, 'per_pizza': 0.5882352941176471},
 {'_id': 7, 'price': 18, 'quantity': 10, 'per_pizza': 0.5555555555555556}]

And if you wanted to find the minimum per pizza price.

In [35]:
result = orders.aggregate([
    {"$project": {"quantity": 1, "price": 1, "per_pizza": {"$divide": ["$quantity", "$price"]}}},   #per_pizza cost = quantity/price
    {"$group": {"_id": 1, "minPerPizza": {"$min": "$per_pizza"}}} #_id 1 is akin to doing Select Min() ... FROM ...
])
list(result)

[{'_id': 1, 'minPerPizza': 0.5263157894736842}]

You could actually obtain the minimum of the per pizza price by using sort and limits as well.

In [36]:
result = orders.aggregate([
    {"$project": {"quantity": 1, "price": 1, "name":1, "per_pizza": {"$divide": ["$quantity", "$price"]}}},   #per_pizza cost = quantity/price
    {"$sort": {"per_pizza": 1}},
    {"$limit": 1}
])
list(result)

[{'_id': 0,
  'name': 'Pepperoni',
  'price': 19,
  'quantity': 10,
  'per_pizza': 0.5263157894736842}]

Lookups can be done on Mongo columns and these are very similar to joins in SQL. A basic example of lookups based on official Mongo example is provided as follows:

In [37]:
db.nuts.insert_many( [
   { "_id" : 1, "item" : "almonds", "price" : 12, "quantity" : 2 },
   { "_id" : 2, "item" : "pecans", "price" : 20, "quantity" : 1 },
   { "_id" : 3  }
] )
db.inventory.insert_many( [
   { "_id" : 1, "sku" : "almonds", "description": "product 1", "instock" : 120 },
   { "_id" : 2, "sku" : "bread", "description": "product 2", "instock" : 80 },
   { "_id" : 3, "sku" : "cashews", "description": "product 3", "instock" : 60 },
   { "_id" : 4, "sku" : "pecans", "description": "product 4", "instock" : 70 },
   { "_id" : 5, "sku": None, "description": "Incomplete" },
   { "_id" : 6 }
] )

In [38]:
results = db.nuts.aggregate( [
   {
     "$lookup":
       {
         "from": "inventory",
         "localField": "item",
         "foreignField": "sku",
         "as": "inventory_docs"
       }
  }
])
list(results)

[{'_id': 1,
  'item': 'almonds',
  'price': 12,
  'quantity': 2,
  'inventory_docs': [{'_id': 1,
    'sku': 'almonds',
    'description': 'product 1',
    'instock': 120}]},
 {'_id': 2,
  'item': 'pecans',
  'price': 20,
  'quantity': 1,
  'inventory_docs': [{'_id': 4,
    'sku': 'pecans',
    'description': 'product 4',
    'instock': 70}]},
 {'_id': 3,
  'inventory_docs': [{'_id': 5, 'sku': None, 'description': 'Incomplete'},
   {'_id': 6}]}]

As you can see, lookup will embed the other document giving you a join like structure. You can use the projection operator to extract relevant fields as follows.

In [39]:
results = db.nuts.aggregate( [
   {
     "$lookup":
       {
         "from": "inventory",
         "localField": "item",
         "foreignField": "sku",
         "as": "inventory_docs"
       },
  },
  {"$project": {"_id":1, "item": 1, "description": {"$arrayElemAt": ["$inventory_docs.description", 0]}}}

])
list(results)

[{'_id': 1, 'item': 'almonds', 'description': 'product 1'},
 {'_id': 2, 'item': 'pecans', 'description': 'product 4'},
 {'_id': 3, 'description': 'Incomplete'}]

This concludes a basic tutorial on the MongoDB. The best resource that you can possibly have to dig deeper is unquestionably the official MongoDB [Manual](https://www.mongodb.com/docs/manual/). Have fun!